# Auto-ablation with Howso Engine

> Note: This feature is experimental and is not universally recommended for production deployment.

## Overview

This notebook provides an overview of using auto-ablation during train to reduce the size of your data _as they are trained_.

In [1]:
import pandas as pd
from pmlb import fetch_data

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## Step 1: Load Data & Feature Mapping

Our example dataset for this recipe is the well-known `Adult` dataset. This dataset works well because it has over 48,000 cases. In general, datasets containing over 25,000 rows work well. The default minimum size is 1,000 cases, below which no cases will be ablated.

This part of the process is identical to other recipes.

In [2]:
df = fetch_data("adult", local_cache_dir="../../../data")
features = infer_feature_attributes(df)

display(df)
display(features.to_dataframe())

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39.0,7,77516.0,9,13.0,4,1,1,4,1,2174.0,0.0,40.0,39,1
1,50.0,6,83311.0,9,13.0,2,4,0,4,1,0.0,0.0,13.0,39,1
2,38.0,4,215646.0,11,9.0,0,6,1,4,1,0.0,0.0,40.0,39,1
3,53.0,4,234721.0,1,7.0,2,6,0,2,1,0.0,0.0,40.0,39,1
4,28.0,4,338409.0,9,13.0,2,10,5,2,0,0.0,0.0,40.0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39.0,4,215419.0,9,13.0,0,10,1,4,0,0.0,0.0,36.0,39,1
48838,64.0,0,321403.0,11,9.0,6,0,2,2,1,0.0,0.0,40.0,39,1
48839,38.0,4,374983.0,9,13.0,2,10,0,4,1,0.0,0.0,50.0,39,1
48840,44.0,4,83891.0,9,13.0,0,1,3,1,1,5455.0,0.0,40.0,39,1


type decimal_places  bounds                        \
                                              min        max allow_null   
age             continuous              0     7.0      148.0       True   
workclass          nominal              0     NaN        NaN      False   
fnlwgt          continuous              0  8103.0  3269017.0       True   
education          nominal              0     NaN        NaN      False   
education-num   continuous              0     1.0       20.0       True   
marital-status     nominal              0     NaN        NaN      False   
occupation         nominal              0     NaN        NaN      False   
relationship       nominal              0     NaN        NaN      False   
race               nominal              0     NaN        NaN      False   
sex                nominal              0     NaN        NaN      False   
capital-gain    continuous              0     0.0   162755.0       True   
capital-loss    continuous              0     0.0     8103.0       True   
hours-per-week  continuous              0     1.0      148.0       True   
native-country     nominal              0     NaN        NaN      False   
target             nominal              0     NaN        NaN      False   

               data_type original_type       
                             data_type size  
age               number       numeric    8  
workclass         number       integer    8  
fnlwgt            number       numeric    8  
education         number       integer    8  
education-num     number       numeric    8  
marital-status    number       integer    8  
occupation        number       integer    8  
relationship      number       integer    8  
race              number       integer    8  
sex               number       integer    8  
capital-gain      number       numeric    8  
capital-loss      number       numeric    8  
hours-per-week    number       numeric    8  
native-country    number       integer    8  
target            number       integer    8

## Step 3: Create Trainee and Set Parameters

The process of creating the `Trainee` is the same as the other recipes. However, after the `Trainee` is created, we set auto-analyze and auto-ablation parameters to ensure that the `Trainee` remains informed as it removes cases during train.

In [3]:
t = Trainee(features=features)

# Set auto-analyze parameters. Note that use_case_weights=True since case-weights are how
# information from ablated cases is retained in the Trainee.
t.set_auto_analyze_params(auto_analyze_enabled=True, use_case_weights=True)

# Set auto-ablation parameters.
t.set_auto_ablation_params(
    auto_ablation_enabled=True,
    influence_weight_entropy_threshold=0.2,
    delta_threshold_map={
        "accuracy": {"target": 0.1}
    }
)

This recipe uses the following parameters for auto-ablation:

- `auto_ablation_enabled` — When this parameter is `False`, auto-ablation will not be performed.
- `influence_weight_entropy_threshold` — This parameter is used to determine which cases will be ablated.
  If the entropy of a case's influence weights would be above this percentile of existing cases' influence
  weight entropies, then the case is ablated. This is set to $0.2$, a very aggressive value, because of the
  next parameter, which acts as a guardrail.
- `delta_threshold_map` — This parameter defines thresholds that are used to determine when ablation should stop.
  In this case, the threshold map instructs auto-ablation to stop if the delta between the current accuracy
  of the `"target"` feature and its previous accuracy is greater than $0.1$. I.e., ablation will stop if accuracy
  for that feature drops from $0.8$ to $0.7$. More formally, ablation will stop if
  $\text{acc}_\text{old} - \text{acc}_\text{new} \geq 0.1$.

## Step 4: Training

Now that the parameters are set, the `Trainee` can be trained as if this were a normal workflow.

In [4]:
t.train(df)

/home/runner/.local/lib/python3.9/site-packages/howso/direct/client.py:374: HowsoWarning: Results may be inaccurate because trainee has not been analyzed with case weights.
Run 'analyze()' with use_case_weights=true, weight_feature='.case_weight' prior to calling this method.
  warnings.warn(w, category=HowsoWarning)


Notably, only a subset of the cases have been trained (the rest have been ablated).

In [5]:
num_trained_cases = t.get_num_training_cases()
num_total_cases = len(df)
num_trained_percent = round((num_trained_cases / num_total_cases) * 100, 3)

print(f"Trained {num_trained_cases:,} cases out of {num_total_cases:,} ({num_trained_percent}%).")

Trained 16,492 cases out of 48,842 (33.766%).


## Step 5: Results

An ablated `Trainee` can be used in much the same way as other `Trainee`s, as long as `use_case_weights=True`. Here, we will investigate the prediction stats for all features.

In [6]:
stats = t.get_prediction_stats(
    action_feature="target",
    details={
        "prediction_stats": True,
        "selected_prediction_stats": ["all"],
    },
)

In [7]:
stats.loc[stats.index != "confusion_matrix", :]

,hours-per-week,education-num,capital-loss,fnlwgt,age,education,target,sex,relationship,native-country,capital-gain,workclass,occupation,marital-status,race
adjusted_smape,20.790643,5.471029,78.686575,43.955446,19.845489,NaN,NaN,NaN,NaN,NaN,97.124022,NaN,NaN,NaN,NaN
mae,7.641018,0.467947,156.246009,77949.004163,7.679788,0.516454,0.170387,0.230631,0.313781,0.151631,1174.126148,0.357196,0.739238,0.222471,0.199484
spearman_coeff,0.463557,0.918288,0.579150,0.121891,0.634067,NaN,NaN,NaN,NaN,NaN,0.605320,NaN,NaN,NaN,NaN
rmse,11.046090,1.070210,417.665699,103652.665532,10.133505,NaN,NaN,NaN,NaN,NaN,5985.940306,NaN,NaN,NaN,NaN
r2,0.179670,0.806578,0.033264,-0.023598,0.409948,NaN,NaN,NaN,NaN,NaN,-0.047240,NaN,NaN,NaN,NaN
smape,21.111668,5.862170,79.240297,43.955589,20.123529,NaN,NaN,NaN,NaN,NaN,97.951151,NaN,NaN,NaN,NaN
precision,NaN,NaN,NaN,NaN,NaN,0.395369,0.81853,0.776997,0.587424,0.100717,NaN,0.425012,0.275879,0.468289,0.465127
recall,NaN,NaN,NaN,NaN,NaN,0.259548,0.788764,0.776422,0.584024,0.067669,NaN,0.253687,0.255907,0.443925,0.218413
mcc,NaN,NaN,NaN,NaN,NaN,0.45283,0.606564,0.553419,0.656737,0.206464,NaN,0.329241,0.25583,0.732529,0.129849
accuracy,NaN,NaN,NaN,NaN,NaN,0.59,0.864,0.79,0.749,0.913968,NaN,0.755756,0.34,0.829,0.859


In [8]:
# The confusion matrix can be retrieved
print("Howso Prediction Results - Confusion Matrix for 'target'")
matrix = pd.DataFrame(stats["target"]["confusion_matrix"]["matrix"])
matrix.index.name = "Predicted"
matrix.columns.name = "Actual"
display(matrix)

Howso Prediction Results - Confusion Matrix for 'target'


Actual,0,1
Predicted,,
0,152,53
1,83,712


## Conclusion

Howso Engine with auto-ablation can compress data to a fraction of its original size while ensuring accuracy (and other measures) remain in an acceptable range without compromising the other capabilities of the `Trainee`.